In [28]:
from importlib import reload

import openforbc.gpu; reload(openforbc.gpu)
import openforbc.gpu.nvidia.gpu; reload(openforbc.gpu.nvidia.gpu)
from openforbc.gpu import GPU
import openforbc.sysfs; reload(openforbc.sysfs);
from openforbc.sysfs.gpu import GPUSysFsHandle

In [29]:
gpus = GPU.get_gpus()
gpu = gpus[0]
print(gpu)

NVIDIA A100-PCIE-40GB


In [30]:
print(gpu.get_supported_types())
print(gpu.get_creatable_types())

[474: GRID A100-1-5C (MIG), 475: GRID A100-2-10C (MIG), 476: GRID A100-3-20C (MIG), 477: GRID A100-4-20C (MIG), 478: GRID A100-7-40C (MIG), 468: GRID A100-4C, 469: GRID A100-5C, 470: GRID A100-8C, 471: GRID A100-10C, 472: GRID A100-20C, 473: GRID A100-40C, 706: GRID A100-1-5CME (MIG)]
[469: GRID A100-5C]


In [38]:
if gpu.get_creatable_types():
    partition = gpu.create_partition(gpu.get_creatable_types()[0])
    print(f"created {partition}")

created 468: GRID A100-4C ca3601d8-195b-4df2-9d2c-119cb46b25c1 on 0000:21:00.4


In [40]:
partitions = gpu.get_partitions()
for p in partitions:
    print(p)

468: GRID A100-4C ca3601d8-195b-4df2-9d2c-119cb46b25c1 on 0000:21:00.4


In [37]:
p = partitions.pop()
print(p)
p.destroy()

469: GRID A100-5C f6913d1f-864f-4e5e-8ef8-68b64eadc44f on 0000:21:00.4


In [38]:
gpu = gpus[0]

sysfs = GPUSysFsHandle.from_gpu(gpu)
print(f"mdev: {sysfs.get_mdev_supported()}, sriov: {sysfs.get_sriov_active()}")
if sysfs.get_sriov_active():
    print(f"numvfs: {sysfs.get_sriov_num_vfs()}")

vgpu_type = gpu.get_creatable_vgpus()[0]

print(f"first available vf is {sysfs.get_sriov_available_vf()}")

for vf in sysfs.get_sriov_vfs():
    if vf.get_mdev_type_available(f"nvidia-{vgpu_type.id}"):
        print(f"{vgpu_type} can be created on {vf._path}")
        break

gpu.create_vgpu(vgpu_type)

mdev: False, sriov: True
numvfs: 16
first available vf is 0000:21:00.7
469: GRID A100-5C can be created on /sys/devices/pci0000:20/0000:20:01.1/0000:21:00.7


469: GRID A100-5C 7ce8ece3-72cd-4f00-abe4-d68ab4468e52 on 0000:21:00.7

In [5]:
from pynvml import *
from pynvml import NVMLError_NotSupported
nvmlInit()

In [5]:
dev = nvmlDeviceGetHandleByIndex(0)
nvmlDeviceGetName(dev)

b'NVIDIA A100-PCIE-40GB'

In [6]:
info = nvmlDeviceGetPciInfo_v3(dev)
info.busIdLegacy

b'0000:21:00.0'

In [25]:
dev = nvmlDeviceGetHandleByPciBusId(b"0000:21:00.0")
nvmlDeviceGetName(dev)

b'NVIDIA A100-PCIE-40GB'

In [10]:
INVALID_GPU_INSTANCE_PROFILE_ID = 0xFFFFFFFF

gips = []
for i in range(NVML_GPU_INSTANCE_PROFILE_COUNT):
    try:
        gips.append(nvmlDeviceGetGpuInstanceProfileInfo(dev, i))
    except NVMLError_NotSupported:
        pass

for id in nvmlDeviceGetSupportedVgpus(dev):
    gi = nvmlVgpuTypeGetGpuInstanceProfileId(id)
    is_mig = gi != INVALID_GPU_INSTANCE_PROFILE_ID
    if is_mig:
        print(f"searching for GIP {gi}")
        gi_info = next(x for x in gips if x.id == gi)
        mem = round(gi_info.memorySizeMB / 1000)
        gi_name = f"{gi_info.sliceCount}g.{mem}gb"

    print(f"{nvmlVgpuTypeGetName(id)} {'(MIG: ' + gi_name +')' if is_mig else ''}")


searching for GIP 19


StopIteration: 

In [16]:
gips[0].memorySizeMB


IndexError: list index out of range

In [1]:
from os import getuid
getuid()

0